# Interface - The Runnable Class
### LangChain Expression Language (LCEL)

---

Alejandro Ricciardi (Omegapy)  
created date: 01/18/2024   
[GitHub](https://github.com/Omegapy)  

Credit: [LangChain](https://python.langchain.com/docs/expression_language/) 

<br>

---

Projects Description:  
**LangChain** is a framework for developing applications powered by language models. 

**In this project:**  
I explore the concept of the [Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) protocol. 

```class langchain_core.runnables.base.Runnable``` 

A unit of work that can be invoked, batched, streamed, transformed and composed.
- ```invoke```/```ainvoke```: Transforms a single input into an output.
- ```batch```/```abatch```: Efficiently transforms multiple inputs into outputs.
- ```stream```/```astream```: Streams output from a single input as it’s produced.
- ```astream_log```: ```Streams``` output and selected intermediate results from an input.

Note: [Pydantic](https://docs.pydantic.dev/latest/) is used to output the schemas, this project uses ```pydantic==1.10.8```

<p></p>
<b style="font-size:15;">
⚠️ This project requires an OpenAi key.
</b>

The input type and output type varies by component:

**Table of Inputs - Outputs**
<table border="1">
  <tr>
    <th>Component</th>
    <th>Input Type</th>
    <th>Output Type</th>
  </tr>
  <tr>
    <td>Prompt</td>
    <td>Dictionary</td>
    <td>PromptValue</td>
  </tr>
  <tr>
    <td>ChatModel</td>
    <td>Single string, list of chat messages or a PromptValue</td>
    <td>ChatMessage</td>
  </tr>
  <tr>
    <td>LLM</td>
    <td>Single string, list of chat messages or a PromptValue</td>
    <td>String</td>
  </tr>
  <tr>
    <td>OutputParser</td>
    <td>The output of an LLM or ChatModel</td>
    <td>Depends on the parser</td>
  </tr>
  <tr>
    <td>Retriever</td>
    <td>Single string</td>
    <td>List of Documents</td>
  </tr>
  <tr>
    <td>Tool</td>
    <td>Single string or dictionary, depending on the tool</td>
    <td>Depends on the tool</td>
  </tr>
</table>

All runnables expose input and output **schemas**. 
To inspect the inputs and outputs: 
- input_schema: an input Pydantic model auto-generated from the structure of the Runnable 
- output_schema: an output Pydantic model auto-generated from the structure of the Runnable

### Project map:
- [API Key](#api-key)
- [Base Example](#base-example)
- [Input Schema](#input-schema)
    - [Chain Schema](#chain-schema-input)
    - [Prompt Schema](#prompt-schema-input)
    - [Model Schema](#model-schema-input)
- [Output Schema](#input-schema)
    - [Chain Schema](#chain-schema-output)
    - [Prompt Schema](#prompt-schema-output)
    - [Model Schema](#model-schema-output)
- [Batch](#batch)
- [Async](async)
    - [Async Stream](async-stream)
    - [Async Invoke](async-invoke)
    - [Async Batch](async-batch)
- [Parallelism](parallelism)
    - [Chains NOT ran in parallel](#chains-not-ran-in-parallel)
    - [Chains ran in parallel](#chains-ran-in-parallel)
- [Parallelism on batches](parallelism-on-batches)
    - [Batches NOT ran in parallel](#batches-not-ran-in-parallel)
    - [Batches ran in parallel](#batches-ran-in-parallel)

<br>

---

### API Key

In [89]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPEN_AI_KEY")

[Project Map](#project-map)

---

### Base Example

In [90]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

chain.invoke({ "topic": "bears" })

AIMessage(content="Why don't bears wear shoes? \n\nBecause they have bear feet!")

[Project Map](#project-map)

---

### Input Schema
A description of the inputs accepted by a Runnable. This is a Pydantic model dynamically generated from the structure of any Runnable. You can call ```.schema()``` on it to obtain a JSONSchema representation.

##### Chain Schema (Input)

In [91]:
# The input schema of the chain is the input schema of its first part, the prompt.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

##### Prompt Schema (Input)

In [92]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

##### Model Schema (Input)

In [93]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'str

[Project Map](#project-map)

---

### Output Schema
A description of the outputs produced by a Runnable. This is a Pydantic model dynamically generated from the structure of any Runnable. You can call ```.schema()``` on it to obtain a JSONSchema representation.

##### Chain Schema (Output)

In [94]:
# The input schema of the chain is the input schema of its first part, the prompt.
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},
   'required': ['content']},
  'HumanMessage': {'title': 'HumanMessage',
   'description': 'A Message from a human.',
   'type': 'object',
   'properties': {'conten

##### Prompt Schema (Output)

In [95]:
prompt.output_schema.schema()

{'title': 'ChatPromptTemplateOutput',
 'anyOf': [{'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': F

##### Model Schema (Output)

In [96]:
model.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},
   'required': ['content']},
  'HumanMessage': {'title': 'HumanMessage',
   'description': 'A Message from a human.',
   'type': 'object',
   'properties': {'conten

[Project Map](#project-map)

---

### Batch

Batch Base code Line

In [97]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

[AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
 AIMessage(content="Sure, here's a cat joke for you:\n\nWhy was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!")]

You can set the number of concurrent requests by using the ```max_concurrency``` parameter ( Max of time that the request can be made to the LLM API server, i.e, if the LLM server does not respond after 'max_concurrency' attends the request will stop )

In [98]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}], config={"max_concurrency": 5})

[AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
 AIMessage(content="Sure, here's a cat joke for you:\n\nWhy don't cats play poker in the wild?\n\nToo many cheetahs!")]

[Project Map](#project-map)

---

### Async

##### Async Stream

In [99]:
async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why don't bears wear shoes?

Because they have bear feet!

##### Async Invoke

In [100]:
await chain.ainvoke({"topic": "bears"})

AIMessage(content="Why don't bears ever wear shoes?\n\nBecause they have bear feet!")

##### Async Batch

In [101]:
await chain.abatch([{"topic": "bears"}])

[AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")]

[Project Map](#project-map)

---

### Parallelism
Let’s take a look at how LangChain Expression Language supports parallel requests. For example, when using a RunnableParallel (often written as a dictionary) it executes each element in parallel.

In [102]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

##### Chains NOT ran in parallel

In [103]:
%%time
chain1.invoke({"topic": "bears"})

CPU times: total: 0 ns
Wall time: 1.23 s


AIMessage(content="Why don't bears like fast food?\n\nBecause they can't catch it!")

In [104]:
%%time
chain2.invoke({"topic": "bears"})

CPU times: total: 0 ns
Wall time: 1.22 s


AIMessage(content="In the wild's embrace,\nGentle giants leave no trace.")

##### Chains ran in parallel

In [105]:
%%time
combined.invoke({"topic": "bears"})

CPU times: total: 0 ns
Wall time: 1.28 s


{'joke': AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
 'poem': AIMessage(content="Gentle giants roam,\nNature's strength, grace, and home.")}

[Project Map](#project-map)

---

### Parallelism on batches
Parallelism can be combined with other runnables. Let’s try to use parallelism with batches.

##### Batches NOT ran in parallel

In [106]:
%%time
chain1.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: total: 0 ns
Wall time: 1.39 s


[AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
 AIMessage(content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!')]

In [107]:
%%time
chain2.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: total: 0 ns
Wall time: 1.6 s


[AIMessage(content="In forest's embrace,\nMajestic bears find solace and grace."),
 AIMessage(content="Whiskers dance, soft paws roam,\nMysterious grace, a feline's home.")]

##### Batches ran in parallel

In [108]:
%%time
combined.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: total: 0 ns
Wall time: 1.5 s


[{'joke': AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
  'poem': AIMessage(content='In forests they roam,\nMajestic bears find their home.')},
 {'joke': AIMessage(content="Why don't cats play poker in the wild?\n\nToo many cheetahs!"),
  'poem': AIMessage(content='Whiskers dance with grace,\nPurring hearts in a feline embrace.')}]